In [5]:
import requests
import urllib
import json
import csv

token = (open("apitoken.txt", "r")).read()
base_url = "https://api.osf.io/v2/"

In [6]:
#function for sending a POST request to the OSF API
def send_post(URL, JSON):
    try:
        response = requests.post(
            url=URL,
            headers={
                "Authorization": "Bearer "+token,
                "Content-Type": "application/vnd.api+json",
            },
            data=JSON,
            allow_redirects=True
        )
        status = response.status_code
        print('URL: {} , Status: {}'.format(url, status))
        if status < 300:
            return response.json()
        else:
            print('Response HTTP Status Code: {status_code}'.format(
                status_code=response.status_code))
            print('Response HTTP Response Body: {content}'.format(
                content=response.content))
    except requests.exceptions.RequestException:
        print('HTTP Request failed')

In [7]:
#function works to create JSON based on given attributes. Tags are fixed for this purpose, 
#but could easily become variables

def create_node(title, descript):
    comp_json = {
        "data": {
            "attributes": {
                "title" : title,
                "category" : "data",
                "description" : descript,
                "tags" : ["not annotated","not transcribed","no first proofread","no second proofread","no third proofread","unassigned"]
            },
            "type": "nodes",
        }
    }    
    return comp_json

print(json.dumps(create_node("title field", "description field")))

{"data": {"type": "nodes", "attributes": {"title": "title field", "description": "description field", "category": "data", "tags": ["not annotated", "not transcribed", "no first proofread", "no second proofread", "no third proofread", "unassigned"]}}}


In [8]:
#this can be run when creating a new node and wanting to add contributors
#takes a list of contributors with each list item being a list with contributor's id and the permissions you want them to have

def post_contributors(node_id, contributors):

    url = "https://api.osf.io/v2/nodes/{}/contributors/".format(node_id)
    print("Adding contributors to url: {}".format(url))

    for contributor in contributors:
        print("Adding contributor ID {} with permission level {} to node ID {}".format(contributor[0], contributor[1], node_id))

        item = {
          "data": {
            "type": "contributors",           # required
            "attributes": {
              "permission": contributor[1]    # optional
            },
            "relationships": {
              "users": {
                "data": {
                  "type": "users",            # required
                  "id":   contributor[0]      # required
                }
              }
            }
          }
        }
                
        json_node = json.dumps(item)
        
        send_post(url, json_node)

    print("{} Contributors Added".format(len(contributors)))

    
    
#contributorList4 = [["2kfwy","write"],["6nywd","write"],["utb5k","write"],["3jzys","admin"],["mr99g","write"]]


#post_contributors("jki34" ,contributorList4)



In [10]:
# ~~~~~DONE RUNNING FOR 1955_1~~~~~~~

# Creates 1 component per line in given CSV nested within the "/children/" URL of given project/component in the OSF.
# Must start with clean CSV
test_url = "https://api.osf.io/v2/nodes/rds64/children/"

erpp_url_1953_1 = "https://api.osf.io/v2/nodes/8uu6m/children/"
erpp_url_1953_2 = "https://api.osf.io/v2/nodes/gydet/children/"
erpp_url_1954_1 = "https://api.osf.io/v2/nodes/tnjfu/children/"
erpp_url_1954_2 = "https://api.osf.io/v2/nodes/49jyb/children/"
erpp_url_1955_1 = "https://api.osf.io/v2/nodes/dgh5e/children/"
erpp_url_1955_2 = "https://api.osf.io/v2/nodes/9a5eh/children/"


#erpp_contributor_list = [["2kfwy","write"],["6nywd","write"],["utb5k","write"],["3jzys","admin"],["mr99g","write"]]

#erpp_contributor_list1 = [['3jzys', 'admin'], ['6nywd', 'admin'], ['utb5k', 'admin'], ['mr99g', 'write'], ['2kfwy', 'write'], ['f77p3', 'write'], ['hhdup', 'write'], ['vkthj', 'write'], ['7595u', 'admin'], ['czszc', 'write'], ['vutud', 'write'], ['bz3n5', 'write'], ['h6zc6', 'write'], ['9j2ay', 'write'], ['5kmjn', 'write']]
erpp_contributor_list2 = [['3jzys', 'admin'], ['6nywd', 'admin'], ['utb5k', 'admin'], ['2kfwy', 'write'], ['f77p3', 'write'], ['hhdup', 'write'], ['vkthj', 'write'], ['7595u', 'admin'], ['czszc', 'write'], ['vutud', 'write'], ['bz3n5', 'write'], ['h6zc6', 'write'], ['9j2ay', 'write'], ['5kmjn', 'write'], ['m6wyb', 'write'], ['mec2f', 'write'], ['kfc3y', 'write'], ['9t3xc', 'write'], ['nnjvk', 'write'], ['tymb5', 'write'], ['xgqtu', 'write'], ['nxfj8', 'write']]

url = erpp_url_1955_2 #edit url based on desired end point

with open('../ERPP/Volume3documents1955_2.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader: #creates a child node for each row in the csv
        #title = row['Doc #']+" "+row['Title'] #if no doc numbers in title - helps when title is absent
        title = row['Title'] #if titles already have doc numbers
        desc = row['Topic']

        print("Title: {}. Description: {}".format(title, desc))    
        #create JSON object

        new_JSON = json.dumps(create_node(title, desc))
        
        #print("Data in JSON as follows: ")
        print(new_JSON)
        
        #send post request with newly created JSON
        res_from_post = send_post(url, new_JSON)
        print(res_from_post)
        
        #use returned json to get id to use to add contributors, then adds contributors
        node_id = res_from_post["data"]["id"]
        post_contributors(node_id, erpp_contributor_list2)

       

Title: 255 ER to Cannon 1955-07-08. Description: military discharges
{"data": {"type": "nodes", "attributes": {"title": "255 ER to Cannon 1955-07-08", "description": "military discharges", "category": "data", "tags": ["not annotated", "not transcribed", "no first proofread", "no second proofread", "no third proofread", "unassigned"]}}}
URL: https://api.osf.io/v2/nodes/9a5eh/children/ , Status: 201
{'data': {'relationships': {'node_links': {'links': {'related': {'href': 'https://api.osf.io/v2/nodes/chw6r/node_links/', 'meta': {}}}}, 'parent': {'links': {'related': {'href': 'https://api.osf.io/v2/nodes/9a5eh/', 'meta': {}}}}, 'view_only_links': {'links': {'related': {'href': 'https://api.osf.io/v2/nodes/chw6r/view_only_links/', 'meta': {}}}}, 'affiliated_institutions': {'links': {'self': {'href': 'https://api.osf.io/v2/nodes/chw6r/relationships/institutions/', 'meta': {}}, 'related': {'href': 'https://api.osf.io/v2/nodes/chw6r/institutions/', 'meta': {}}}}, 'contributors': {'links': {'re

In [10]:
#Optional data cleaning 
import re

p = re.compile('\d\d\d\s')

#find obvious errors in title
if p.match(title) is None:
            print(title)

In [69]:
with open('../ERPP/Volume3documents1954_2.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader: #creates a child node for each row in the csv
        #docNum = row['Doc #'] 
        title = row['Doc #']+" "+row['Title'] 
        desc = row['Topic']

        print("Title: {}. Description: {}".format(title, desc))    
        #create JSON object
        
        

Title: 160 Murray to ER 1954-06-01. Description: Brown v. Board
Title: 161 My Day 1954-06-04. Description: MD red scare, Oppenheimer
Title: 162 Remarks on Welfare State 1954-06-05. Description: welfare state
Title: 163 ER to Marxer 1954-06-08. Description: science, atomic energy, 
Title: 164 ER to the Harrimans 1954-06-21. Description: Wiltwyck
Title: 165 ER to Florence Willert 1954-06-24. Description: British colonialism
Title: 166 ER to Auguston 1954-06-25. Description: communists in other countries--USSR
Title: 167 ER to Eiler 1954-06-28. Description: red scare
Title: 168 ER to Benson 1954-06-28. Description: Hiss, 
Title: 169 Statement on Cancelled trip to the USSR 1954-07-01. Description: Russia trip
Title: 170 ER to Pauli Murray 1954-07-07. Description: ER running for office (to Pauli Murray)
Title: 171 ER to Congressman? 1954-07-08. Description: Dinosaur National Monument/ National Parks
Title: 172 ER to Zimmerman 1954-07-13. Description: parochial schools
Title: 173 ER to "Gent

In [57]:
#testing code

new_JSON = json.dumps(create_node("Testing_API_title", "Description for testing API"))

#print("Data in JSON as follows: ")
#print(new_JSON)

#send post request with newly created JSON
#res_of_postcall = send_post(url, new_JSON)
#json_from_postcall2 = json.dumps(json_from_postcall)

node_id = json_from_postcall["data"]["id"]
print(node_id)
test_contri = [["hsey5","write"],["r8dsa","admin"]]
post_contributors(node_id, test_contri)

qczq4
Adding contributors to url: https://api.osf.io/v2/nodes/qczq4/contributors/
Adding contributor ID hsey5 with permission level write to node ID qczq4
URL: https://api.osf.io/v2/nodes/rbs4g/children/ , Status: 201
Adding contributor ID r8dsa with permission level admin to node ID qczq4
URL: https://api.osf.io/v2/nodes/rbs4g/children/ , Status: 201
2 Contributors Added
